In [1]:
import easydict
import cv2
import numpy as np
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
import os
import time
from PIL import ImageFont, ImageDraw, Image

C:\Users\pc\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\pc\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\pc\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\pc\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1)

In [2]:
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

In [3]:
model = load_model("./CNN_best_model.h5")

ValueError: ('Unrecognized keyword arguments:', dict_keys(['ragged']))

In [14]:
# TF-Pose, HOG descriptor 객체 생성 
e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(432, 368))
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

[2021-01-28 14:26:28,220] [TfPoseEstimator] [INFO] loading graph from C:\Users\pc\Desktop\2021_UGRP\personal\Nwoo\Gesture_Recognition\main_work\models\graph/mobilenet_thin/graph_opt.pb(default size=432x368)


In [1]:
count = 0
cap = cv2.VideoCapture("./Test_video.mp4")
while True:
    ret, image = cap.read()
    people_bbox = dict()
    
    if not ret:
        break
    # Bounding Box 그리기
    detected, _ = hog.detectMultiScale(image)
    
    for (x,y,w,h) in detected:    
        # 가로 200pixel 세로 300pixel 이하 제외
        if w < 200 and h < 300: continue

        cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),3)
        
        # image resizing and reshape (Preprocessing)
        input_image = image[int(y):int(y+h), int(x):int(x+w)]
        img = Image.open(input_image)
        img = img.convert("RGB")
        img = img.resize((500,500))
        data = np.asarray(img)
        
        X = np.array(data)
        X = X.astype("float") / 256
        X = X.reshape(-1, 500,500,3)
        
        # model prediction
        pred = model.predict(X)
        people_bbox[(x,y,w,h)] = pred
        
    # Skeleton 그리기 with Background
    humans = e.inference(image, upsample_size=4.0)
    image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
    
    # esc 누르면 종료
    if cv2.waitKey(10) == 27:                    
        break
    print('%d.jpg done' % count)
    count += 1
    
    # Class name print 
    for (x,y,w,h) in people_bbox:
        gesture = people_bbox[(x,y,w,h)]
        cv2.putText(image, gesture, (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2, cv2.LINE_AA)
    
    cv2.imshow("Gesture_Recognition", image)
#윈도우 종료
cap.release()
cv2.destroyWindow('Gesture_Recognition')

NameError: name 'cv2' is not defined